In [16]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 2)
    
    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 8 * 8)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x
# Define the dataset and data loader
train_folder = 'D:/Data/Train'
test_folder = 'D:/Data/Test'
transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])
train_dataset = datasets.ImageFolder(train_folder, transform=transform)
test_dataset = datasets.ImageFolder(test_folder, transform=transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
]))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Instantiate the neural network and define the loss function and optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the neural network
for epoch in range(15):
    model.train()  # Set the model to training mode
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Forward pass
        scores = model(data)
        loss = criterion(scores, targets)
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print training progress
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{15}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')
    
    # Evaluate the neural network on the test set
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        correct = 0
        total = 0
        for data, targets in test_loader:
            scores = model(data)
            _, predicted = torch.max(scores.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        accuracy = 100 * correct / total
        print(f'Test Accuracy: {accuracy:.2f}%')



Epoch [1/15], Batch [0/2962], Loss: 0.6982
Epoch [1/15], Batch [100/2962], Loss: 0.6883
Epoch [1/15], Batch [200/2962], Loss: 0.6836
Epoch [1/15], Batch [300/2962], Loss: 0.5350
Epoch [1/15], Batch [400/2962], Loss: 0.7061
Epoch [1/15], Batch [500/2962], Loss: 0.7066
Epoch [1/15], Batch [600/2962], Loss: 0.7033
Epoch [1/15], Batch [700/2962], Loss: 0.6846
Epoch [1/15], Batch [800/2962], Loss: 0.6861
Epoch [1/15], Batch [900/2962], Loss: 0.5573
Epoch [1/15], Batch [1000/2962], Loss: 0.7253
Epoch [1/15], Batch [1100/2962], Loss: 0.5712
Epoch [1/15], Batch [1200/2962], Loss: 0.6081
Epoch [1/15], Batch [1300/2962], Loss: 0.6843
Epoch [1/15], Batch [1400/2962], Loss: 0.6251
Epoch [1/15], Batch [1500/2962], Loss: 0.5949
Epoch [1/15], Batch [1600/2962], Loss: 0.6631
Epoch [1/15], Batch [1700/2962], Loss: 0.6274
Epoch [1/15], Batch [1800/2962], Loss: 0.6788
Epoch [1/15], Batch [1900/2962], Loss: 0.6118
Epoch [1/15], Batch [2000/2962], Loss: 0.6106
Epoch [1/15], Batch [2100/2962], Loss: 0.5209


In [17]:
torch.save(model.state_dict(), 'person_detection_model_v3_74.pth')
print('Модель создана')

Модель создана
